Created by Pooja Subramaniam and Marc Aurel Vischer on Tue, May 8.

Temperature is given as a tuple of daily high and low value, both in degrees Celsius as ints.

Precipitation is given as "probability" as float.

Wind is given as a tuple of strength in Bft (int) and direction (e.g. "NE" if wind _comes from_ north east).

In [23]:
import urllib3
from bs4 import BeautifulSoup

In [24]:
#These are the urls referring directly to high, low temperature
hi_lo_url = "https://wetter.bild.de/web2014/ifr-wetter-deutschland.asp"
prec_url = "https://wetter.bild.de/web2014/ifr-niederschlag-deutschland.asp"
wind_url = "https://wetter.bild.de/web2014/ifr-windstaerken-deutschland.asp"

In [15]:
#load and parse page
http = urllib3.PoolManager()
hi_lo_bs = BeautifulSoup(http.request('GET', hi_lo_url).data, "lxml")
prec_bs = BeautifulSoup(http.request('GET',prec_url).data, "lxml")
wind_bs = BeautifulSoup(http.request('GET',wind_url).data, "lxml")
#print(hi_lo.prettify())

/home/bccn2017/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/home/bccn2017/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/home/bccn2017/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [25]:
#TEMPERATURE HIGH/LOW, bild has today + 5 days forecast for that
#iterate over days, extract day layer for each
temp_dicts = []
for day in range(5):
    # extract current day layer
    day_layer = hi_lo_bs.find_all('div', id="wk_layer_wr{}".format(day))
    #print(day_layer[0]['id'])
    if len(day_layer)!=1:
        raise Exception("Found more than one layer for single day.")
        
    # extract all the cities from that layer
    day_cities = day_layer[0].find_all('div', class_="wk_map_text")
    day_dict = {}
    for city in day_cities:
        hi_lo_str = city.nobr.nextSibling.nextSibling
        high = int(hi_lo_str.split('|')[0].split('°')[0])
        low = int(hi_lo_str.split('|')[1].split('°')[0])
        day_dict[city.nobr.string] = (high, low)
    temp_dicts.append(day_dict)

In [27]:
#PRECIPITATION,  bild has only today + 2 days forecast for that
#iterate over days, extract day layer for each
prec_dicts = []
for day in range(1,4): #layer 0 corresponds to next 6 hrs, layer 1 to entire current day
    # extract current day layer
    day_layer = prec_bs.find_all('div', id="wk_layer_wr{}".format(day))
    #print(day_layer[0]['id'])
    if len(day_layer)!=1:
        raise Exception("Found more than one layer for single day.")
        
    # extract all the cities from that layer
    day_cities = day_layer[0].find_all('div', class_="wk_map_text")
    day_dict = {}
    for city in day_cities:
        prec_str = city.nobr.nextSibling.nextSibling
        prec_value = int(prec_str.split()[0])/100
        day_dict[city.nobr.string] = prec_value
    prec_dicts.append(day_dict)

In [28]:
#WIND,  bild again has today + 5 days forecast
WIND_GER_ENG = {"w":"W", "nw":"NW", "n":"N", "no":"NE", "o":"E", "so":"SE", "s":"S", "sw":"SW"}
#iterate over days, extract day layer for each
wind_dicts = []
for day in range(5):
    # extract current day layer
    day_layer = wind_bs.find_all('div', id="wk_layer_wr{}".format(day))
    #print(day_layer[0]['id'])
    if len(day_layer)!=1:
        raise Exception("Found more than one layer for single day.")
        
    # extract all the cities from that layer
    day_cities = day_layer[0].find_all('div', class_="wk_map_text")
    day_dict = {}
    for city in day_cities:
        wind_str = city.nobr.nextSibling.nextSibling
        wind_strength = int(wind_str.split()[0])
        wind_symbol_url = city.parent.img['src']
        wind_direction_raw = wind_symbol_url.split('.')[0].split('/')[-1]
        wind_direction = WIND_GER_ENG[wind_direction_raw]
        day_dict[city.nobr.string] = (wind_strength,wind_direction)
    wind_dicts.append(day_dict)